# Establishing a Connection - Alice
## Role: Inviter

In this notebook we'll be initiating the aries DID Exchange protocol. Before an agent can begin to interact with another agent they must engage in this protocol resulting in the exchange of Decentralised Identifiers (DIDs) and corresponding DID Documents. The information within these documents can then be used to establish a secure communication channel between the parties.

For a detailed description of this protocol it is highly recommened to read this [aries-rfc]((https://github.com/hyperledger/aries-rfcs/tree/master/features/0023-did-exchange).

This notebook has the following phases:

1. Pull in dependencies
2. Instantiate the controller for the aries agent (See the docker-compose.yml)
3. Set up a listener for basicmessages events emitted by the controller when it receives a webhook
4. Use the controller to create an invitation from our agent
5. Copy the invitation output from 4 and move over to Bob's notebook

<b>Carry on in [Bob's notebook](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>
    
11. Accept Request for Connection
12. Send Trust Ping to Bob

    



### 1. Pull in dependencies

In [8]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [9]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)


In [10]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

### 3. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [11]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


### 4. Use the controller to create an invitation from our agent

An invitation is a JSON object, as shown below, it contains the relevant information required for another agent to connect with it and exchange identifiers. This object must be passed to the agent Alice wishes to connect to out of band, in this instance we will just copy it across.

In [12]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection b21aed1f-2dd6-41d8-b0de-bcf46f64acc8 in State invitation
Connection ID b21aed1f-2dd6-41d8-b0de-bcf46f64acc8
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '9676a279-d978-42bf-9e51-757f038b36d4', 'label': 'Alice', 'serviceEndpoint': 'http://192.168.65.3:8020', 'recipientKeys': ['Bve459nP9qUssiivfn7MMjPpxp5RhNzgyFfTTE61db2e']}


### 5. Copy the invitation output from 4 and move over to the [Bob notebook](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb) 

### 11. Accept Request for Connection

In [6]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 441463fe-987e-4943-a6c3-60fbd00c50f3 in State response
ACCEPT REQUEST
{'invitation_key': '2ZfKo22KWUudXnBq6WsCnHh2op3VkoCWyugHEd6WyCZn', 'connection_id': '441463fe-987e-4943-a6c3-60fbd00c50f3', 'their_label': 'Carol', 'accept': 'manual', 'state': 'response', 'invitation_mode': 'once', 'rfc23_state': 'response-sent', 'my_did': 'L1s7TR6CiAMVpR2skQCbfU', 'routing_state': 'none', 'their_role': 'invitee', 'their_did': 'Dr9aLJnoB8RpnWz1EDYDzy', 'updated_at': '2023-01-02 07:03:59.440652Z', 'created_at': '2023-01-02 07:03:10.692617Z'}
state response


### 12. Send Trust Ping to activate the conneciton

In [7]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '0dccb6d4-52ed-4b5e-ab1a-a9f65aec2958'}
Connection Handler Called
Connection 441463fe-987e-4943-a6c3-60fbd00c50f3 in State active


####  12.1 Check if connection is active

In [5]:
connection_id = '441463fe-987e-4943-a6c3-60fbd00c50f3'
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])

{'rfc23_state': 'completed', 'their_did': 'Dr9aLJnoB8RpnWz1EDYDzy', 'created_at': '2023-01-02 07:03:10.692617Z', 'state': 'active', 'connection_id': '441463fe-987e-4943-a6c3-60fbd00c50f3', 'accept': 'manual', 'invitation_key': '2ZfKo22KWUudXnBq6WsCnHh2op3VkoCWyugHEd6WyCZn', 'their_label': 'Carol', 'updated_at': '2023-01-02 07:04:05.808478Z', 'my_did': 'L1s7TR6CiAMVpR2skQCbfU', 'routing_state': 'none', 'their_role': 'invitee', 'invitation_mode': 'once'}
Is Active? active


## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [7]:
response = await agent_controller.terminate()
print(response)

Terminating webhooks listener failed! RuntimeError('Site <aiohttp.web_runner.TCPSite object at 0xffffb7d16420> is not registered in runner <aiohttp.web_runner.AppRunner object at 0xffffb4140bd0>') occurred.


None
